# Molecular dynamics simulations with the Lennard-Jones potential

## Setup

The first thing you will need to do is install the [ASE package](https://wiki.fysik.dtu.dk/ase/) (for Atomic Simulation Environment).  First, make sure you are in the correct Python virtual environment.  If you have been installing packages with `pip`, you can just run:

    pip install ase

and it should install it into the correct environment.  Make sure it works by running the cell below:

In [ ]:
import ase

If you get errors at this stage, please ask for help.

**Note for Anaconda/conda users**: The ASE package is not available through Anaconda's default distribution channels.  It is, however, available from [conda-forge](https://conda-forge.org/packages/).  You can use the following installation command (again, make sure you are in the correct environment):

    conda install -c conda-forge --strict-channel-priority ase

and try the `import` command above.

## Extra steps for Google Colab

In [ ]:
!pip install ase

The installation command below is needed to make sure the `nglview` viewer widget works in the Colab environment.  Do not modify it unless you know what you are doing!

In [ ]:
!pip install nglview ipywidgets==7.5

You will be prompted to restart the notebook runtime.  Don't worry, your files won't be deleted and the notebook won't be changed, but you will need to re-run any `import` statements.

Do not re-run the install commands!

Finally, you will need to run the cell below to enable the viewer "widget" to display in the Colab notebook.  Make sure to re-run this every time you restart the notebook runtime or reload the notebook itself.

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

**Note**: The `nglviewer` widget is still unreliable in the Colab notebook environment (this is mostly Google's fault, rather than the fault of the widget developers.  It's one of the many reasons I would discourage using Colab in the first place... but if you're reading this, you're probably stuck using it for one reason or another.)

If the widget does not display properly (e.g. missing atoms or unit cell, or missing sliders/controls, or controls not working properly), try _closing the widget_ by _clearing the cell output_ (open the menu to the top left of the widget and then click "Clear selected outputs").  **Make sure** to close the widget after every use so that it doesn't interfere with other widgets that you open later on!

### Other imports

In [ ]:
import ase
import ase.io

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

### Viewing atomic structures

It is important that we visualize atomic structures and their evolution in time, so that we can _see_ the (qualitative) behaviour of a system.  ASE provides a few ways to do this.  The most full-featured and user-friendly one is the [ASE GUI](https://wiki.fysik.dtu.dk/ase/ase/gui/gui.html), accessible from the command line with:

    ase gui <structure_file.xyz>

where `<structure_file.xyz>` is a file containing the structures you want to view.

You can also view structures directly in the notebook environment.  This can be very useful for quick checking of structures without interrupting your workflow.  To do this, you first have to import ASE's `view()` function:

In [ ]:
from ase.visualize import view

Now, if you just call `view(structure)` on an atomic structure, you will see a visualization directly in the notebook.  Note that this visualization is still a bit basic; you can get more features by installing the `nglview` plugin:

    pip install nglview
or

    conda install -c conda-forge --strict-channel-priority nglview

You can now call the viewer function like: `view(structure, viewer='ngl')` and you will get a more interactive structure viewer.  Try it out below!

## Initial configuration

Let's start with an _initial configuration_ of atoms.  I've prepared a starting structure (a crystal of 32 Ar atoms in the FCC close-packed configuration) using the code below, but you can just load the structure from the file included with this tutorial.

In [ ]:
from ase.build import bulk, make_supercell
starting_unit = bulk('Ar', 'fcc', a=5.0, cubic=True)
starting_config = make_supercell(starting_unit, 2*np.eye(3, dtype=int))

In [ ]:
import ase.io

In [ ]:
ase.io.write('starting_config.xyz', starting_config)

----

In [ ]:
starting_config = ase.io.read('starting_config.xyz')

In [ ]:
# Note: This may not work if you haven't gotten nglview installed or configured correctly
view(starting_config, viewer='ngl')

## Defining the potential

Now, we will define the _potential energy surface_ that determines how the atoms interact.  In some ways, this is the most important ingredient in an atomistic simulation!

As discussed, we will use the Lennard-Jones potential here, defined by the equation:
$$
u(r) = 4\varepsilon \left(\left(\frac{r_0}{r}\right)^{12} - \left(\frac{r_0}{r}^{6}\right)\right)
$$
applied between all pairs of atoms within some _maximum distance_ of each other.  This maximum distance is another important parameter of the potential, and it is typically called the "cutoff".

To calculate this potential for any atomic configuration, we will use ASE's built-in potential _calculator_:

In [ ]:
from ase.calculators.lj import LennardJones

In [ ]:
?LennardJones

Take a moment to check out the documentation.  This is also available online in the [ASE documentation page](https://wiki.fysik.dtu.dk/ase/ase/calculators/others.html#lennard-jones).

We will need to initialize it with a few parameters -- besides the cutoff radius, we need values for $\varepsilon$ and $r_0$.  Let's use the values proposed in this publication: [J. A. White, _J. Chem. Phys._ **111**, 9352-9355 (1999)]().

The proposed values are: $r_0 = 3.345\,\text{Å}$ and $\varepsilon = 125.7 k_B$, where $k_B$ is Boltzmann's constant.

But hang on -- we need to make sure to convert these to the units that ASE uses!  Luckily, ASE already uses Ångström for length units.  For the energy units, we just use the `ase.units` module to find the appropriate conversion:

In [ ]:
ase.units.kB

So, putting it together, we initialize the potential as follows:

In [ ]:
lj_argon_calc = LennardJones(sigma=3.345, epsilon=125.7*ase.units.kB, rc=10.0, smooth=True)

(I've somewhat arbitrarily chosen a cutoff radius of 10 Å here because that's the length of our cell.  You can play around with different cutoff values later, although be warned that the effect on the results can be quite subtle.)

## Doing something with the potential

Now that we've defined the potential, let's do something useful with it!  The simplest thing would be an energy minimization, where we try to find the structure that minimizes the energy of this potential energy surface.  Note that this structure will generally depend on the potential and its parameters!

In [ ]:
starting_config.set_calculator(lj_argon_calc)

In [ ]:
from ase.optimize import BFGS

In [ ]:
from ase.filters import ExpCellFilter

In [ ]:
# Slightly annoying extra step we need in order to optimize the cell (volume) in addition to the positions
atoms_cell = ExpCellFilter(starting_config, hydrostatic_strain=True)
opt = BFGS(atoms_cell)
opt.run(fmax=0.005)

In [ ]:
starting_config

### Question

What is the _cell parameter_ of the resulting, optimized structure?  How does it compare to the experimental value (for solid argon)?

In [ ]:
starting_config.cell[0,0] / 2

## Run dynamics

Okay, now we get to the fun part!  Let's run some actual simulations and see the atoms move!

In [ ]:
import ase.md

First, we need an "integrator" that actually solves the equations of motion.  Velocity Verlet is a good choice:

In [ ]:
from ase.md import MDLogger

In [ ]:
vv = ase.md.VelocityVerlet(atoms=starting_config, timestep=1.0, trajectory='trial_traj.traj')
vv.attach(MDLogger(vv, starting_config, 'md.log', header=True, stress=False,
          peratom=False, mode="a"), interval=1)

And now we just run it for 50 timesteps and see what happens!

In [ ]:
vv.run(50)

Finally, try loading the output in the `ase gui` and see if we got anything...

In [ ]:
traj = ase.io.read('trial_traj.xyz', '-50:')

In [ ]:
view(traj[10], viewer='x3d')

----
SPOILER ALERT



...nothing happened.  That's because we haven't initialized the _velocities_ on the atoms, so they're not moving!  Let's set them up with a nice initial, "thermal" velocity.

In [ ]:
from ase.md import velocitydistribution

In [ ]:
velocitydistribution.MaxwellBoltzmannDistribution(starting_config, temperature_K=40)

In [ ]:
vv.run(50)

In [ ]:
import numpy as np

In [ ]:
md_log = np.loadtxt('md.log', skiprows=52)

In [ ]:
md_log

In [ ]:
time, e_tot, e_pot, e_kin, T = md_log.T

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.plot(time, e_pot, c='C0', label='Potential energy')
plt.plot(time, e_kin, c='C1', label='Kinetic energy')
plt.plot(time, e_tot, c='k', label='Total energy')
plt.legend()
plt.xlabel('Time / ps')
plt.ylabel('Energy / eV')

In [ ]:
plt.plot(time, T)
plt.xlabel('Time / ps')
plt.ylabel('Temperature / K')

## Constant-temperature dynamics with the NVT ensemble

In [ ]:
nvt = ase.md.Langevin(atoms=starting_config, timestep=1.0, trajectory='traj_nvt.traj',
                      temperature_K=200, friction=0.1 / ase.units.fs)

In [ ]:
nvt.attach(MDLogger(nvt, starting_config, 'md_nvt.log', header=True, stress=True,
           peratom=False, mode="w"), interval=10)

In [ ]:
nvt.run(500)

In [ ]:
starting_config

In [ ]:
md_log_nvt = np.loadtxt('md_nvt.log', skiprows=1)

In [ ]:
time, e_tot, e_pot, e_kin, T = md_log_nvt.T[:5]

In [ ]:
stress_components = md_log_nvt.T[5:]

In [ ]:
plt.plot(time, e_pot, c='C0', label='Potential energy')
plt.plot(time, e_kin, c='C1', label='Kinetic energy')
plt.plot(time, e_tot, c='k', label='Total energy')
plt.legend()
plt.xlabel('Time / ps')
plt.ylabel('Energy / eV')

In [ ]:
plt.plot(time, T)
plt.xlabel('Time / ps')
plt.ylabel('Temperature / K')

In [ ]:
pressure = -1.0 / 3 * np.sum(stress_components[:3], axis=0)
plt.plot(time, pressure)
plt.xlabel('Time / ps')
plt.ylabel('Pressure / GPa')

## Constant pressure and temperature (NPT)

First, let's reset the starting configuration...

In [ ]:
starting_config = ase.io.read('starting_config.xyz')

In [ ]:
lj_argon_calc = LennardJones(sigma=3.345, epsilon=125.7*ase.units.kB, rc=10.0, smooth=True)

In [ ]:
starting_config.calc = lj_argon_calc

In [ ]:
from ase.md.nptberendsen import NPTBerendsen

Now, we need the compressibility to determine good parameters for the barostat.  We might not have direct measurements of the compressibility, but we can derive it from the speed of sound.

The CRC handbook says the speed of sound in liquid argon at the boiling point, about 83 K, is 838.3 m/s (and the NIST tables agree).  So let's use that and hope it doesn't change too much when we get to the solid phase.  (We don't have to be _too_ exact here, it's just to set the timescale of the barostat.)

In [ ]:
vsound_aunits = 840 * ase.units.m / ase.units.s

In [ ]:
vsound_aunits

In [ ]:
ase.units.Ang

In [ ]:
den_aunits = np.sum(starting_config.get_masses()) / starting_config.get_volume()

In [ ]:
den_aunits

In [ ]:
# Compressibility is one over the bulk modulus, which is the speed of sound squared times the density
comp_aunits = 1.0 / (vsound_aunits**2 * den_aunits)

In [ ]:
comp_aunits

In [ ]:
velocitydistribution.MaxwellBoltzmannDistribution(starting_config, temperature_K=120)

In [ ]:
npt = NPTBerendsen(atoms=starting_config, timestep=1.0, trajectory='traj_npt.traj', loginterval=10,
                   pressure_au=1*ase.units.bar, temperature_K=120,
                   taut=100*ase.units.fs, taup=500*ase.units.fs, compressibility_au=comp_aunits)

In [ ]:
npt.attach(MDLogger(npt, starting_config, 'md_npt.log', header=True, stress=True,
           peratom=False, mode="w"), interval=10)

In [ ]:
npt.run(2000)

In [ ]:
md_log_npt = np.loadtxt('md_npt.log', skiprows=1)

In [ ]:
time, e_tot, e_pot, e_kin, T = md_log_npt.T[:5]

In [ ]:
stress_components = md_log_npt.T[5:]

In [ ]:
plt.plot(time, e_pot, c='C0', label='Potential energy')
plt.plot(time, e_kin, c='C1', label='Kinetic energy')
plt.plot(time, e_tot, c='k', label='Total energy')
plt.legend()
plt.xlabel('Time / ps')
plt.ylabel('Energy / eV')

In [ ]:
plt.plot(time, T)
plt.xlabel('Time / ps')
plt.ylabel('Temperature / K')

In [ ]:
pressure = -1.0 / 3 * np.sum(stress_components[:3], axis=0)
plt.plot(time, pressure)
plt.xlabel('Time / ps')
plt.ylabel('Pressure / GPa')

In [ ]:
npt_traj = ase.io.read('traj_npt.traj', ':')

In [ ]:
cell_volumes = np.array([frame.get_volume() for frame in npt_traj])

In [ ]:
plt.plot(time, cell_volumes)
plt.xlabel('Time / ps')
plt.ylabel('Cell volume / Å^3')

### Questions

Please record your answers in this notebook by creating new cells below.
 
 - Is the simulation _equilibrated_ after this initial runtime of about 50 ps?  If not, adjust the runtime (in `run()` above) and run it longer.
 - What is the _density_ that the Lennard-Jones potential (with the given parameters) predicts for these conditions?  Is this realistic?  Why / why not?  _(Note: You may find it useful to use `ase.units` to convert the density into units you are more familiar with)_
 - Find an experimental reference for the density of liquid argon under similar conditions.  How good is the agreement?

### Task

Keep increasing the temperature of the thermostat until the system enters the gas phase.  (How do you know?)

At what temperature does this happen?  See if you can pin down the phase transition temperature to within 10 K or so.

**Question**: Does this agree with the phase transition (vaporization) temperature from the reference literature?  If not, how bad is the disagreement?  Discuss with someone next to you what the possible sources of such a disagreement could be.

## Diffusion coefficient

Now, let's return to the liquid-phase simulation at 120 K, 1 bar, using the potential we had before.

In [ ]:
starting_config = ase.io.read('starting_config.xyz')

In [ ]:
lj_argon_calc = LennardJones(sigma=3.345, epsilon=125.7*ase.units.kB, rc=10.0, smooth=True)

In [ ]:
starting_config.calc = lj_argon_calc

And let's again run an NPT simulation to _equilibrate_ the system to the given thermodynamic conditions.  We will later _turn off_ the thermostat and barostat to avoid interfering with the detailed dynamical correlations that determine the diffusion coefficient.

In [ ]:
velocitydistribution.MaxwellBoltzmannDistribution(starting_config, temperature_K=85)

In [ ]:
npt = NPTBerendsen(atoms=starting_config, timestep=1.0, trajectory='traj_diff_eq.traj', loginterval=10,
                   pressure_au=1*ase.units.bar, temperature_K=85,
                   taut=100*ase.units.fs, taup=500*ase.units.fs, compressibility_au=comp_aunits)

In [ ]:
npt.attach(MDLogger(npt, starting_config, 'md_diff_eq.log', header=True, stress=True,
           peratom=False, mode="w"), interval=10)

In [ ]:
npt.run(2000)

Check we've equilibrated:

In [ ]:
md_log_npt = np.loadtxt('md_diff_eq.log', skiprows=1)

In [ ]:
time, e_tot, e_pot, e_kin, T = md_log_npt.T[:5]

In [ ]:
stress_components = md_log_npt.T[5:]

In [ ]:
plt.plot(time, e_pot, c='C0', label='Potential energy')
plt.plot(time, e_kin, c='C1', label='Kinetic energy')
plt.plot(time, e_tot, c='k', label='Total energy')
plt.legend()
plt.xlabel('Time / ps')
plt.ylabel('Energy / eV')

In [ ]:
plt.plot(time, T)

In [ ]:
eq_traj = ase.io.read('traj_diff_eq.traj', ':')

In [ ]:
cell_volumes = np.array([frame.get_volume() for frame in npt_traj])

In [ ]:
plt.plot(time, cell_volumes[:201])
plt.xlabel('Time / ps')
plt.ylabel('Cell volume / Å^3')

And now we need to run an NVE simulation with our old friend, Velocity Verlet:

In [ ]:
vv = ase.md.VelocityVerlet(atoms=starting_config, timestep=1.0, trajectory='diff_traj.traj')
vv.attach(MDLogger(vv, starting_config, 'diff_md.log', header=True, stress=False,
          peratom=False, mode="a"), interval=10)

Let's run it for about the same amount of time:

In [ ]:
vv.run(5000)

### Task

Load the trajectory from the file and _write a program_ to compute the mean squared displacement from the atomic positions.

You may wish to start with only one atom, and track its mean squared displacement in all three dimensions, then sum them up to get the MSD for that atom only.  Then, you'll want to average this over all the atoms.

Plot the MSD against time.  Do you see a linear trend?  Try to figure out its slope.

Next, try to find an experimental value for the diffusion coefficient in liquid argon.  Is the value you computed close?